# Equal-Weight S&P 500 Index Fund

## Introduction

The goal of this Notebook is to evenly distribute the investment across a list of stocks, so that you can determine how many shares of each stock you can buy with your allocated amount per stock.

By evenly distributing your total investment and calculating how many shares you can buy with each allocation, you manage your investment risk and ensure a diversified portfolio.

## Installing and importing required libraries

In [44]:
# pip install yfinance

In [45]:
import yfinance as yf
import pandas as pd
import numpy as np

## Load the list of SP500 stocks list

Here, I downloaded it fromt the internet. We can also do web scrpaing to get the data from any page.

In [46]:
# Load the S&P 500 stock symbols from the CSV file
sp500_stocks = pd.read_csv('sp_500_stocks.csv')

## Get the stock data for the SP500 list

Use the yfinance python library to get the required data.

In [47]:
# Initialize an empty list to store the results
data = []

# Iterate through each stock symbol in the CSV file
for symbol in sp500_stocks['Ticker']:
    try:
        # Fetch the stock data using yfinance
        stock = yf.Ticker(symbol)
        stock_info = stock.info

        # Get the stock price and market cap
        stock_price = stock_info.get('currentPrice', None)
        market_cap = stock_info.get('marketCap', None)

        # Append the data to the list
        data.append([symbol, stock_price, market_cap, 0])

    except Exception as e:
        print(f"Failed to get data for {symbol}: {e}")
        data.append([symbol, None, None, 0])

# Create a DataFrame from the collected data
data = pd.DataFrame(data, columns=['Ticker', 'Stock Price', 'Market Capitalization', 'Stocks to Buy'])

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=eaz4RUsjmKr
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=eaz4RUsjmKr
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=eaz4RUsjmKr
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.

In [48]:
data

,Ticker,Stock Price,Market Capitalization,Stocks to Buy
0,A,136.90,3.994208e+10,0
1,AAL,9.86,6.475101e+09,0
2,AAP,60.21,3.589859e+09,0
3,AAPL,216.24,3.287735e+12,0
4,ABBV,189.93,3.354809e+11,0
...,...,...,...,...
500,YUM,136.79,3.846056e+10,0
501,ZBH,108.08,2.201071e+10,0
502,ZBRA,319.95,1.650315e+10,0
503,ZION,45.95,6.786677e+09,0


Check for `None` values.

In [49]:
print(data.isna().sum())

Ticker                    0
Stock Price              42
Market Capitalization    44
Stocks to Buy             0
dtype: int64


Check the stocks with `None` rows.

In [50]:
# Find rows where any value is NaN
rows_with_nan = data[data.isna().any(axis=1)]

# Print rows with NaN values
print(rows_with_nan)

    Ticker  Stock Price  Market Capitalization  Stocks to Buy
5      ABC          NaN                    NaN              0
6     ABMD          NaN                    NaN              0
28    ALXN          NaN                    NaN              0
39    ANTM          NaN                    NaN              0
48    ATVI          NaN                    NaN              0
61    BF.B          NaN                    NaN              0
68     BLL          NaN                    NaN              0
71   BRK.B          NaN                    NaN              0
88    CERN          NaN                    NaN              0
107    COG          NaN                    NaN              0
118    CTL          NaN                    NaN              0
121   CTXS          NaN                    NaN              0
124    CXO          NaN                    NaN              0
135  DISCA          NaN                    NaN              0
136  DISCK          NaN                    NaN              0
137   DI

Create a copy of the original dataframe so that we don't manipulate the original dataframe.

In [70]:
final_df = data.copy(deep=True)

Drop `None` values.

In [71]:
# Assuming final_df is your DataFrame
final_df = final_df.dropna()

# Reset the index and drop the old index column
final_df = final_df.reset_index(drop=True)

In [72]:
print(final_df.isna().sum())
rows_with_nan = final_df[final_df.isna().any(axis=1)]
print(rows_with_nan)

Ticker                   0
Stock Price              0
Market Capitalization    0
Stocks to Buy            0
dtype: int64
Empty DataFrame
Columns: [Ticker, Stock Price, Market Capitalization, Stocks to Buy]
Index: []


## Calculating the Number of Shares to Buy

This represents the total amount of money you have to invest. In this case, it's $100,000.

In [74]:
portfolio_size = 100000

You want to evenly distribute your total investment amount (portfolio_size) across all the stocks.
- `len(final_df.index)` gives you the number of stocks in your DataFrame.
- `position_size` is the amount of money you will allocate to each stock. It's calculated by dividing the total portfolio size by the number of stocks

In [75]:
position_size = portfolio_size/len(final_df.index)

For each stock, you want to determine how many shares you can buy with the allocated amount (`position_size`).

`position_size // final_df['Stock Price']` calculates the number of shares you can buy of each stock with the allocated amount.

In [76]:
final_df['Stocks to Buy'] = position_size // final_df['Stock Price']

## Final Output

In [77]:
final_df

,Ticker,Stock Price,Market Capitalization,Stocks to Buy
0,A,136.90,3.994208e+10,1.0
1,AAL,9.86,6.475101e+09,21.0
2,AAP,60.21,3.589859e+09,3.0
3,AAPL,216.24,3.287735e+12,1.0
4,ABBV,189.93,3.354809e+11,1.0
...,...,...,...,...
456,YUM,136.79,3.846056e+10,1.0
457,ZBH,108.08,2.201071e+10,2.0
458,ZBRA,319.95,1.650315e+10,0.0
459,ZION,45.95,6.786677e+09,4.0


From the above dataframe, we can see the column `Stocks to Buy` which tells us the number of stocks to buy with allocatde portfolio in SP500 stocks.